In [1]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model

In [2]:
df_sample = pd.read_csv('C://Users//snuist//Data//Lending_Club_Data_Subsample.csv', engine = 'python')

In [3]:
df_X = df_sample.iloc[:,1:26].as_matrix()
df_y = df_sample[['bad']].as_matrix()

In [7]:
# axis = 0 -> Row 기준으로 잘라라
# 800번째 위로는 train, 밑으로는 test set으로 만들자
[X_train, X_test] = np.split(df_X, [800], axis=0)

In [8]:
X_train.shape

(800, 25)

In [9]:
X_test.shape

(200, 25)

In [10]:
[y_train, y_test] = np.split(df_y, [800], axis=0)

In [13]:
# 다중공선성을 배제하기 위하여 intercept제거
mdl_OLS = linear_model.LinearRegression(fit_intercept = 'false')
mdl_OLS.fit(X_train, y_train)
mdl_OLS.coef_

array([[ -1.74463383e-01,  -1.38167873e-01,  -2.10968788e-01,
          6.49442023e-01,  -1.25841973e-01,   3.69306365e-03,
         -3.69306365e-03,  -3.76116823e-02,   3.16016196e-03,
          3.44515204e-02,   1.53022693e-02,  -1.53022693e-02,
          9.74744201e-06,  -2.83940208e-04,  -2.56287362e-07,
          1.69647658e-03,  -3.22204404e-03,   2.20679766e-02,
          1.04964672e-03,  -1.29939896e-01,  -9.12059370e-05,
          1.94849379e-01,  -1.85217912e-03,   7.10182468e-11,
          2.27926607e-03]])

In [14]:
mdl_Lasso = linear_model.Lasso(fit_intercept = 'false', alpha = 10**(-3))
mdl_Lasso.fit(X_train, y_train)
mdl_Lasso.coef_

array([ -3.03089095e-02,   0.00000000e+00,  -6.25167863e-02,
         0.00000000e+00,   7.57674945e-03,   0.00000000e+00,
        -0.00000000e+00,  -3.61014346e-02,   0.00000000e+00,
         2.64630586e-02,   2.03105924e-02,  -2.32934283e-17,
         8.42155008e-06,  -2.52103000e-04,  -2.24444613e-07,
         1.77500964e-03,  -3.00844512e-03,   2.39599998e-02,
         6.68555079e-04,  -1.74744206e-02,   1.44426649e-04,
         5.62418902e-02,  -1.78697924e-03,   7.09970706e-11,
         2.71775660e-03])

In [19]:
y_train_OLS = mdl_OLS.predict(X_train)
np.mean((y_train - y_train_OLS)**2) # in-sample fit of OLS

0.10682318582011309

In [26]:
y_train_Lasso = mdl_Lasso.predict(X_train)[:,np.newaxis]
np.mean((y_train - y_train_Lasso)**2)

0.10787275231393881

## 왜 [:,np.newaxis]를 도입할까?

In [30]:
mdl_Lasso.predict(X_train).shape

(800,)

In [31]:
mdl_Lasso.predict(X_train)[:,np.newaxis].shape
# 이렇게 해야 (800,1)로 되어 곱할 수 있다.

(800, 1)

In [22]:
y_test_OLS = mdl_OLS.predict(X_test)
np.mean((y_test - y_test_OLS)**2)

0.12545107795979948

In [27]:
# Test set에서는 라쏘가 나음(overfitting막아주기)

y_test_Lasso = mdl_Lasso.predict(X_test)[:,np.newaxis]
np.mean((y_test - y_test_Lasso)**2)

0.12446861491834732

## cross validation으로 람다를 정하기

In [42]:
# cv가 나눈 덩어리의 수
mdl_LassoCV = linear_model.LassoCV(cv = 4)
mdl_LassoCV.fit(X_train, y_train.ravel())

y_test_LassoCV = mdl_LassoCV.predict(X_test)[:,np.newaxis]
np.mean((y_test - y_test_LassoCV)**2)

0.12800624999999999

In [43]:
mdl_LassoCV.alpha_

1675485.4506218736

In [44]:
mdl_LassoCV.coef_

array([ -0.00000000e+00,   0.00000000e+00,  -0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,  -0.00000000e+00,
         0.00000000e+00,  -0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,  -0.00000000e+00,
         0.00000000e+00,  -0.00000000e+00,  -0.00000000e+00,
         0.00000000e+00,  -0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,  -0.00000000e+00,   6.43550202e-26,
         0.00000000e+00])